In [1]:
pip install evaluate

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install rouge-score


In [7]:
import torch
from PIL import Image
from datasets import load_dataset
import tqdm
import evaluate
from lavis.models import load_model_and_preprocess
device = "cuda" if torch.cuda.is_available() else "cpu"

In [9]:
torch.cuda.is_available()

True

In [11]:
import torch
from PIL import Image
from datasets import load_dataset
import tqdm
import evaluate
from lavis.models import load_model_and_preprocess

device = "cuda" if torch.cuda.is_available() else "cpu"
model, vis_processors, txt_processors = load_model_and_preprocess(
    name="blip2_opt", 
    model_type="pretrain_opt2.7b", 
    is_eval=True, 
    device=device
)

dataset = load_dataset("flaviagiammarino/path-vqa")
eval_split = dataset["validation"]

bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")

predictions = []
references = []
exact_match = 0

def generate_prompt(question: str) -> str:
    return f"Question: {question} Answer:"

for sample in tqdm.tqdm(eval_split, desc="Evaluating"):
    image = sample["image"].convert("RGB")
    original_question = sample["question"]
    ref_answer = sample["answer"]
    
    prompt = generate_prompt(original_question)
    
    processed_image = vis_processors["eval"](image).unsqueeze(0).to(device)
    inputs = {
        "image": processed_image,
        "prompt": prompt
    }
    
    outputs = model.generate(inputs, max_length=50)
    pred_answer = outputs[0].strip()
    
    predictions.append(pred_answer)
    references.append(ref_answer)
    
    if pred_answer.lower() == ref_answer.lower():
        exact_match += 1

accuracy = exact_match / len(eval_split)
bleu_result = bleu_metric.compute(predictions=predictions, references=references)
rouge_result = rouge_metric.compute(predictions=predictions, references=references)

print("Evaluation Results on PathVQA (Validation):")
print(f"Accuracy (Exact Match): {accuracy:.4f}")
print("BLEU:", bleu_result)
print("ROUGE:", rouge_result)

D:\Conda\envs\lavis\lib\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
D:\Conda\envs\lavis\lib\site-packages\lavis\models\eva_vit.py:433: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_sa

Evaluation Results on PathVQA (Validation):
Accuracy (Exact Match): 0.2770
BLEU: {'bleu': 0.0, 'precisions': [0.111331557167961, 0.0006691201070592171, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.566696269982238, 'translation_length': 17641, 'reference_length': 11260}
ROUGE: {'rouge1': 0.28604461653194335, 'rouge2': 0.00022034692509209208, 'rougeL': 0.2860672893197276, 'rougeLsum': 0.28571932566399305}
